<a href="https://colab.research.google.com/github/tedblackson/DeepLearning/blob/main/DL_Lab3_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Name: Tewodros Alemu
##ID: UGR/7621/12





In [3]:
import torch
class DenseLayer:
  def __init__(self, features, nth_neurons):
    self.weights = 0.01 * torch.rand(features, nth_neurons)
    self.bias = torch.zeros((1, nth_neurons))

  def forward(self, inputs):
    self.output = torch.matmul(inputs, self.weights) + self.bias

In [4]:
class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + torch.exp(-inputs))

In [5]:
class Activation_ReLU:
    def forward(self, inputs):
        self.output = torch.max(torch.zeros_like(inputs), inputs)


In [6]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = torch.exp(inputs - torch.max(inputs, dim=1, keepdim=True).values)
        probabilities = exp_values / torch.sum(exp_values, dim=1, keepdim=True)
        self.output = probabilities

In [7]:
class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        num_samples = y_pred.shape[0]
        y_pred_clipped = torch.clip(y_pred, 1e-7, 1 - 1e-7)
        log_likelihoods = -torch.log(y_pred_clipped[range(num_samples), y_true])
        return log_likelihoods.mean()

In [8]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, neurons_per_layer, output_size):
        self.layers = []
        self.activations = []

        # Input layer
        self.layers.append(DenseLayer(input_size, neurons_per_layer))
        self.activations.append(Activation_ReLU())

        # Hidden layers
        for _ in range(hidden_layers):
            self.layers.append(DenseLayer(neurons_per_layer, neurons_per_layer))
            self.activations.append(Activation_ReLU())

        # Output layer
        self.layers.append(DenseLayer(neurons_per_layer, output_size))
        self.activations.append(Activation_Softmax())

    def forward(self, inputs):
        for layer, activation in zip(self.layers, self.activations):
            layer.forward(inputs)
            activation.forward(layer.output)
            inputs = activation.output

        self.output = inputs

    def predict(self, inputs):
        self.forward(inputs)
        return torch.argmax(self.output, dim=1)

    def calculate_loss(self, inputs, targets):
        self.forward(inputs)
        loss = Loss_CategoricalCrossentropy()
        return loss.forward(self.output, targets)

    def calculate_accuracy(self, inputs, targets):
        predictions = self.predict(inputs)
        correct = torch.sum(predictions == targets).item()
        accuracy = correct / len(targets)
        return accuracy


In [9]:
input_size = 4
hidden_layers = 3
neurons_per_layer = 18
output_size = 3


In [10]:
model = NeuralNetwork(input_size, hidden_layers, neurons_per_layer, output_size)


In [11]:
inputs = torch.randn(100, input_size)
targets = torch.randint(0, output_size, (100,))
model.forward(inputs)


In [12]:
loss = model.calculate_loss(inputs, targets)
accuracy = model.calculate_accuracy(inputs, targets)

print("Loss:", loss)
print("Accuracy:", accuracy)

Loss: tensor(1.0986)
Accuracy: 0.29
